<a href="https://colab.research.google.com/github/Mathias0210/Bach/blob/main/Pyth2fors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
!pip install mlxtend

In [5]:
from google.colab import files
uploaded = files.upload()
df = pd.read_excel('SalesData.xlsx')
# show the first few rows
df.head()

Saving SalesData.xlsx to SalesData (7).xlsx


,TransaktionsID,RækkeID,KundeID,Navn,Adresse1,Postnr,LevBynavn,Land,ExtSystemId,KategoriID,ProduktID,ProductName,Uom
0,21231914,75288197,22775,ATEA A/S,Kabling Vest,8200.0,Århus N.,DK,iphone,Bump Cap,4368750532,3M First Base 3 Bump Cap 2030609 Classic 25mm ...,STK
1,21231914,75288198,22775,ATEA A/S,Kabling Vest,8200.0,Århus N.,DK,iphone,Pandelygter,4441431029,LEDLENSER Solidline SH2,STK
2,21231914,75288199,22775,ATEA A/S,Kabling Vest,8200.0,Århus N.,DK,iphone,"1,5V",4440610181,Duracell Procell batteri AAA 10-PAK,PAK
3,21211932,75286996,37230,Frederikshavn Forsyning A/,NaN,9900.0,Frederikshavn,DK,lemu-dk,Tilbehør,0411100214,Kabelbøjle 12-16 mm,STK
4,21211932,75287008,37230,Frederikshavn Forsyning A/,NaN,9900.0,Frederikshavn,DK,lemu-dk,Adapter/Stik,9911004934,"Isoleret formgods for SF6 anlæg, sæt for 3 faser",STK


In [6]:
# Prepend '_' to ProduktID
df['ProduktID'] = df['ProduktID'].apply(lambda x: '_'+str(x))

#Sætter alle produktnavne til stort
df['ProductName']=df['ProductName'].str.upper()

In [8]:
# DataFrame for building the recommendation system
orders = df[['TransaktionsID', 'ProduktID']]
print(orders.shape)
print(orders.head(10))
# DataFrame for retrieving product descriptions
products = df[['ProduktID', 'ProductName']].copy()

print(products.shape)
print(products.head(10))

(10000, 2)
   TransaktionsID    ProduktID
0        21231914  _4368750532
1        21231914  _4441431029
2        21231914  _4440610181
3        21211932  _0411100214
4        21211932  _9911004934
5        21211932  _9911000239
6        21211932  _9911004918
7        21211932  _9911002651
8        21211932  _9911002402
9        21211932  _9911004934
(10000, 2)
     ProduktID                                        ProductName
0  _4368750532  3M FIRST BASE 3 BUMP CAP 2030609 CLASSIC 25MM ...
1  _4441431029                            LEDLENSER SOLIDLINE SH2
2  _4440610181                DURACELL PROCELL BATTERI AAA 10-PAK
3  _0411100214                                KABELBØJLE 12-16 MM
4  _9911004934   ISOLERET FORMGODS FOR SF6 ANLÆG, SÆT FOR 3 FASER
5  _9911000239                           KRYMPEMUFFE 4X6,0-16 MM²
6  _9911004918             ENDEAFSLUTNING IXSU-F3121-12KV1L 25-95
7  _9911002651                  KRYMPEENDEMUFFE Ø35-70 MM 102L048
8  _9911002402                  KRYMPEENDEM

In [9]:
# Drop duplicated products
products = products[~products.duplicated()]


print(products.shape)
print(products.head(10))

(6477, 2)
      ProduktID                                        ProductName
0   _4368750532  3M FIRST BASE 3 BUMP CAP 2030609 CLASSIC 25MM ...
1   _4441431029                            LEDLENSER SOLIDLINE SH2
2   _4440610181                DURACELL PROCELL BATTERI AAA 10-PAK
3   _0411100214                                KABELBØJLE 12-16 MM
4   _9911004934   ISOLERET FORMGODS FOR SF6 ANLÆG, SÆT FOR 3 FASER
5   _9911000239                           KRYMPEMUFFE 4X6,0-16 MM²
6   _9911004918             ENDEAFSLUTNING IXSU-F3121-12KV1L 25-95
7   _9911002651                  KRYMPEENDEMUFFE Ø35-70 MM 102L048
8   _9911002402                  KRYMPEENDEMUFFE Ø25-50 MM 102L044
10  _0411101569                                   TAG CP3 MONTERET


In [48]:
# Fra tidligere trin er der nu kun ca. 6500 linjer i produkt men det bemærkes 

products.to_excel('Products.xlsx')
files.download('Products.xlsx')

#Det bemærkes at forekllige productnavne har samme produktID, ved nærmere inspektion er det dog de samme produkter bare med forskellige navne.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
#Fjerner duplikerede linjer med productID og beholder


products = (products
.sort_values(by=['ProduktID'],ascending=False))

products.head(160)

,ProduktID,ProductName
2683,_RIVS810,"RUSTFRIE STÅLRØR EN 1.4301 GLØDEDE 8X1,00 MM"
268,_RIVG4832,"RUSTFRIE GEVINDRØR EN 1.4301 GLØDEDE 48,3X3,25 MM"
2917,_RIV4603,"RUSTFRIE STÅLRØR EN 1.4301 60X60X3,00 MM"
5665,_RIV4404,"RUSTFRIE STÅLRØR EN 1.4301 40X40X4,00 MM"
2916,_RIV4403,"RUSTFRIE STÅLRØR EN 1.4301 40X40X3,00 MM"
...,...,...
5545,_9978864311,VENTILATOR EDM 100 HZ
9999,_9978863079,SILENT 200 CZ
2014,_9978858000,THERMEX SILENT ONE Ø100 HVID
5107,_9978858000,"THERMEX SILENT ONE, HVID"


In [54]:
#Fjerner fejlene fra ProdutData ved at overskrive dubletterne

products = products.drop_duplicates(subset='ProduktID', keep="last")
print(products.shape)
products.head(160)

(6351, 2)


,ProduktID,ProductName
2683,_RIVS810,"RUSTFRIE STÅLRØR EN 1.4301 GLØDEDE 8X1,00 MM"
268,_RIVG4832,"RUSTFRIE GEVINDRØR EN 1.4301 GLØDEDE 48,3X3,25 MM"
2917,_RIV4603,"RUSTFRIE STÅLRØR EN 1.4301 60X60X3,00 MM"
5665,_RIV4404,"RUSTFRIE STÅLRØR EN 1.4301 40X40X4,00 MM"
2916,_RIV4403,"RUSTFRIE STÅLRØR EN 1.4301 40X40X3,00 MM"
...,...,...
5545,_9978864311,VENTILATOR EDM 100 HZ
9999,_9978863079,SILENT 200 CZ
5107,_9978858000,"THERMEX SILENT ONE, HVID"
9951,_9978857386,SILENT 100 CHRZ - DESIGN


In [12]:
#Der blevv fjernet lidt over 100 fejl med denne metode og jeg må antage at produkterne er de samme og tabellen nu er klar til brug


#Jeg går nu videre til min liste af transaktioner
#Et loop der laver en liste med alle produktid'er for hver transaktion

def string_list(x):
    return [str(i) for i in x]

orders = orders.groupby('TransaktionsID')['ProduktID'].apply(list).reset_index()

#Der var sneget sig et par ekstra [] ind i produktname som jeg her fjerner

df['ProduktID'] = df['ProduktID'].str.strip('[]')
orders.head(100)


,TransaktionsID,ProduktID
0,16458869,[_1016008150]
1,16671499,[_3980020020]
2,16686990,"[_4419003986, _4438001708, _728428023]"
3,17632461,[_980419066]
4,18029570,[_7818230573]
...,...,...
95,21051238,[_7897992599]
96,21052330,[_7897700998]
97,21052385,"[_4333956718, _4333956890, _4333956719, _43299..."
98,21053098,"[_4361904104, _5788180022, _RIVS810]"


In [113]:
#orders.to_excel('orders.xlsx')
#files.download('orders.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
#Importerer mlxtend https://towardsdatascience.com/mlxtend-a-python-library-with-interesting-tools-for-data-science-tasks-d54c723f89cd

from mlxtend.preprocessing import TransactionEncoder

In [16]:
# fit the TransactionEncoder
te = TransactionEncoder()

te.fit(orders['ProduktID'])

# do the transformation
orders_1hot = te.transform(orders['ProduktID'])

#Omdanner til en matrix med produktID som kolonner og transaktioner som rækker.
orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
orders_1hot.head()

,_000090104,_000090106,_000090108,_000090110,_000090126,_000090404,_000090406,_000092104,_000092106,_000092108,...,_RGKF303,_RIG14832,_RIHF3320,_RIV4253,_RIV4303,_RIV4403,_RIV4404,_RIV4603,_RIVG4832,_RIVS810
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [115]:
#Importerer apriori som er vorres unsupervised metode
from mlxtend.frequent_patterns import apriori

#Instantierer apriori med datasettet der er klargjort og giver første anbefalinger

ap = apriori(orders_1hot, min_support=0.01, max_len=5, use_colnames=True)
ap.head(6)

,support,itemsets
0,0.011673,(_1017036338)
1,0.010376,(_1017054433)
2,0.010052,(_1017056266)
3,0.011349,(_1017056279)
4,0.010376,(_1017056402)


In [116]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(ap,metric="lift", min_threshold = 0.0001)
print(rules)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []


In [102]:
#Funktion der laver foreslag baseret på produktID

def predict(antecedent, rules, max_results= 6):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]
  

    preds = predict({'_1017036338'}, rules)
    preds

    # Display the descriptions of the predictions
    for produktID in preds:  
      print(products[produktID])  